In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import cross_validation, metrics 
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
with open('./data/train.json') as json_data:
    d = json.load(json_data)


In [15]:
nn = len(d['price'])
index=d['listing_id']
apDF = pd.DataFrame(d, index=index)

In [30]:
apDF.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
110554,2.0,3,f7fc4fd7b6b80615ebfce3e212e17cce,2016-04-03 02:22:45,This one WON'T LAST!! Here is a stunning 3 Bed...,Hausman Street,"[Pre-War, Dishwasher, Hardwood Floors]",medium,40.7245,6818139,-73.9389,198d2e96429920ff71cd06ddff323713,[https://photos.renthop.com/2/6818139_a50e80ff...,2995,23 Hausman Street
11542,1.0,0,45c480f3a0771a4e6ddf7a81790a0b07,2016-06-01 02:46:47,Exposed brick wall w/gas electric & heat inclu...,E 78 Street,"[Loft, Laundry in Unit, Hardwood Floors, Dogs ...",medium,40.7706,7088908,-73.9501,136b844f49c4eae017d6cabe8ca03b9d,[https://photos.renthop.com/2/7088908_f931ca0d...,1950,503 E 78 Street
11543,1.0,0,b0d419ddd5ff150f96b328c61ee6e7e2,2016-06-07 17:49:07,Fully furnished studio with kitchenette and fu...,W 83rd St,"[Furnished, Short Term Allowed]",high,40.7855,7122047,-73.9752,0a4de95e82c122b78494dcea6cb9f44d,[https://photos.renthop.com/2/7122047_de461dbe...,1850,155 W 83rd St
11540,1.0,3,c3d5bcc96eb5d960b9a3feb67e71115b,2016-06-16 05:00:55,3 bedrooms <br />2 big queen sized bds...,40th St,[],low,40.6444,7169671,-73.9936,324631e1aacaa41bc4475f879756bbb6,[https://photos.renthop.com/2/7169671_c5963a80...,2400,973 40th St
11541,1.0,3,3989b2e8d5eb6cd4ccfc8fa4257b277b,2016-06-16 03:44:57,"Comfortable spacious renovated apartment, with...",W 144th Beautiful Spacious Renovated Secure L...,"[Cats Allowed, Dogs Allowed, No Fee, Reduced F...",medium,40.8260,7168565,-73.9519,2a4e0d2668fde3afa267d312b3a2a9d8,[https://photos.renthop.com/2/7168565_617723df...,2500,612 W 144th


In [16]:
categories = [1,2,3]

In [17]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(apDF['features'], 
                                                                                     apDF['interest_level'], 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [18]:
def convert_data(dataSeries):
    dataStr = dataSeries.apply(lambda x: str(x))
    dataStr = dataStr.values
    return(dataStr)

In [19]:
test_dataV=convert_data(test_data)
print len(test_dataV)

14806


In [20]:
test_labelsV = convert_data(test_labels)
print len(test_labelsV)

14806


In [21]:
train_labelsV = convert_data(train_labels)
#train_labelsV = train_labels.apply(lambda x: str(x))
#train_labelsV = train_labelsV.values
#type(train_labelsV[0])
print len(train_labelsV)

34546


In [22]:
train_dataV = convert_data(train_data)
#train_dataV = train_data.apply(lambda x: str(x))
#train_dataV = train_dataV.values
#type(train_dataV[0])
print len(train_dataV)

34546


In [23]:
#replace words with their counts(occurances) in descriptions
from sklearn.feature_extraction.text import CountVectorizer

X_train_counts = count_vect.fit_transform(train_dataV)
print X_train_counts.shape

X_test_counts = count_vect.transform(test_dataV)
print X_test_counts.shape

(34546, 970)
(14806, 970)


In [25]:
print count_vect.vocabulary_.get(u'allowed')


81


In [39]:
featureNames = count_vect.get_feature_names()
data = X_train_counts.toarray()
data = pd.DataFrame(data)
data.columns = featureNames
print data.iloc[2][data.iloc[2] !=0]

balcony       1
building      1
center        1
dishwasher    1
doorman       1
fee           1
fitness       1
floors        1
hardwood      1
in            2
laundry       2
no            1
outdoor       1
pool          1
space         1
swimming      1
terrace       1
unit          1
Name: 2, dtype: int64


In [37]:
train_data.head()

36084                 [Pre-War, Dogs Allowed, Cats Allowed]
40448                         [Dishwasher, Hardwood Floors]
119507    [Swimming Pool, Balcony, Doorman, Fitness Cent...
87386     [Doorman, Fitness Center, No Fee, Dogs Allowed...
73867                         [Dishwasher, Hardwood Floors]
Name: features, dtype: object

In [40]:
#transform occurances to frequences
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print X_train_tf.shape
#same for test data
tf_transformerTest = TfidfTransformer(use_idf=False).fit(X_test_counts)
X_test_tf = tf_transformerTest.transform(X_test_counts)
print X_test_tf.shape

(34546, 970)
(14806, 970)


In [44]:
dataFr = X_train_tf.toarray()
dataFr = pd.DataFrame(dataFr)
dataFr.columns = featureNames
print dataFr.iloc[0][dataFr.iloc[0] !=0]

allowed    0.707107
cats       0.353553
dogs       0.353553
pre        0.353553
war        0.353553
Name: 0, dtype: float64


In [101]:
#predict using Multinomial bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_tf, train_labelsV)
predicted = clf.predict(X_train_tf)
metrics.accuracy_score(train_labelsV, predicted)

0.69461008510391942

In [102]:
predicted = clf.predict(X_test_tf)
print len(predicted)
print len(test_labelsV)

14806
14806


In [103]:
metrics.accuracy_score(test_labelsV, predicted)

0.69552883965959744

In [105]:
#try to use SGD SVM and PipeLine
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)),])


In [108]:
_ = text_clf.fit(train_dataV, train_labels)
predicted = text_clf.predict(test_dataV)
np.mean(predicted == test_labelsV)         


0.6962717817101175

In [109]:
type(X_train_tf)

scipy.sparse.csr.csr_matrix

In [112]:
#transform all text features into counts
count_vect = CountVectorizer(token_pattern="[A-Za-z-]{3,}")
textFeatures = convert_data(apDF['features'])
features_counts = count_vect.fit_transform(textFeatures)


#print features_counts.vocabulary_.get(u'allowed')

In [113]:
#transform all text features into frequencies
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(features_counts)
X_features_tf = tf_transformer.transform(features_counts)

dataFr = X_features_tf.toarray()
dataFr = pd.DataFrame(dataFr)
featureNames = count_vect.get_feature_names()
dataFr.columns = featureNames
print dataFr.iloc[0][dataFr.iloc[0] !=0]



dishwasher    0.5
floors        0.5
hardwood      0.5
pre-war       0.5
Name: 0, dtype: float64


In [114]:
dfSimple = apDF[['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price', 'interest_level']]
interests={'low':1, 'medium':2, 'high':3}
levels = dfSimple['interest_level']
levels = levels.apply(lambda x: interests[x])
dfSimple['InterestLevel'] = levels
dfSimple.head()


C:\Users\battlann\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,bathrooms,bedrooms,latitude,longitude,price,interest_level,InterestLevel
110554,2.0,3,40.7245,-73.9389,2995,medium,2
11542,1.0,0,40.7706,-73.9501,1950,medium,2
11543,1.0,0,40.7855,-73.9752,1850,high,3
11540,1.0,3,40.6444,-73.9936,2400,low,1
11541,1.0,3,40.8260,-73.9519,2500,medium,2


In [115]:
dfSimple = dfSimple.reset_index()
dfSimple = dfSimple.drop('index', 1)
dfSimple.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,InterestLevel
0,2.0,3,40.7245,-73.9389,2995,medium,2
1,1.0,0,40.7706,-73.9501,1950,medium,2
2,1.0,0,40.7855,-73.9752,1850,high,3
3,1.0,3,40.6444,-73.9936,2400,low,1
4,1.0,3,40.8260,-73.9519,2500,medium,2


In [116]:
dfWithFeatures = pd.concat([dfSimple, dataFr], axis=1)
dfWithFeatures.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,InterestLevel,-greal,-hour,-square,...,wrap,xae,xoxo,xxl,xxlarge,yard,year,yoga,york,your
0,2.0,3,40.7245,-73.9389,2995,medium,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0,40.7706,-73.9501,1950,medium,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0,40.7855,-73.9752,1850,high,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,3,40.6444,-73.9936,2400,low,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,3,40.8260,-73.9519,2500,medium,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
dfWithFeatures = dfWithFeatures.drop('interest_level', 1)

In [118]:
#try to random forest with text features
from sklearn.ensemble import RandomForestClassifier

In [119]:
X = dfWithFeatures.ix[:, dfWithFeatures.columns != 'InterestLevel']
y =  dfWithFeatures['InterestLevel']
print X.head()
print y.head()

   bathrooms  bedrooms  latitude  longitude  price  -greal  -hour  -square  \
0        2.0         3   40.7245   -73.9389   2995     0.0    0.0      0.0   
1        1.0         0   40.7706   -73.9501   1950     0.0    0.0      0.0   
2        1.0         0   40.7855   -73.9752   1850     0.0    0.0      0.0   
3        1.0         3   40.6444   -73.9936   2400     0.0    0.0      0.0   
4        1.0         3   40.8260   -73.9519   2500     0.0    0.0      0.0   

   abraham  absolute  ...   wrap  xae  xoxo  xxl  xxlarge  yard  year  yoga  \
0      0.0       0.0  ...    0.0  0.0   0.0  0.0      0.0   0.0   0.0   0.0   
1      0.0       0.0  ...    0.0  0.0   0.0  0.0      0.0   0.0   0.0   0.0   
2      0.0       0.0  ...    0.0  0.0   0.0  0.0      0.0   0.0   0.0   0.0   
3      0.0       0.0  ...    0.0  0.0   0.0  0.0      0.0   0.0   0.0   0.0   
4      0.0       0.0  ...    0.0  0.0   0.0  0.0      0.0   0.0   0.0   0.0   

   york  your  
0   0.0   0.0  
1   0.0   0.0  
2   0.0 

In [120]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(X.as_matrix(), 
                                                                                     y, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [121]:
rf = RandomForestClassifier(n_estimators=150, random_state = 0)
rf.fit(train_data, train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [122]:
preds = rf.predict(test_data)

In [124]:
print metrics.mean_absolute_error(test_labels, preds)
print metrics.accuracy_score(test_labels, preds)

0.335269485344
0.707618533027
